In [1]:
%load_ext autoreload
%autoreload 2


import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

In [2]:
import os
from PIL import Image

# Directory path


# Counter for images found

def transformed_images_address(idx,base_path):
# Iterate through folders
    image_count = 0
    image_paths = []

    # Iterate through folders
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            image_path = os.path.join(folder_path, f"{idx}.png")
            if os.path.exists(image_path):

                #image = Image.open(image_path)
                image_paths.append(image_path)
                image_count += 1
                #print(f"Processing {image_path}")
            #else:
            #   print(f"No found in {folder_path}")

    # Print the total count
    return image_count,image_paths


base_path = "/work/images/images"
image_count,image_paths = transformed_images_address(1,base_path)
print(image_count)
#print(image_paths)

299


In [3]:




# from transformers import pipeline, AutoProcessor
# from PIL import Image    
# import requests

# model_id = "llava-hf/llava-1.5-7b-hf"
# pipe = pipeline("image-to-text", model=model_id)
# url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/ai2d-demo.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# # Define a chat history and use `apply_chat_template` to get correctly formatted prompt
# # Each value in "content" has to be a list of dicts with types ("text", "image") 
# conversation = [
#     {
#       "role": "user",
#       "content": [
#           {"type": "text", "text": "What does the label 15 represent? (1) lava (2) core (3) tunnel (4) ash cloud"},
#           {"type": "image"},
#         ],
#     },
# ]
# processor = AutoProcessor.from_pretrained(model_id)

# prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

# outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
# print(outputs)
# {"generated_text": "\nUSER: What does the label 15 represent? (1) lava (2) core (3) tunnel (4) ash cloud\nASSISTANT: Lava"}

In [4]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True,
    device_map='auto'
)

processor = AutoProcessor.from_pretrained(model_id)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


In [5]:
def predict(question,image_file):

    # Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
    # Each value in "content" has to be a list of dicts with types ("text", "image") 
    conversation = [
        {

        "role": "user",
        "content": [
            {"type": "text", "text": question},
            {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    
    raw_image = Image.open(image_file).convert("RGB")
    inputs = processor(images=raw_image, text=prompt, return_tensors='pt')

    output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    answer = processor.decode(output[0][2:], skip_special_tokens=True)
    return answer

In [6]:
question = "Tell me more about this picture?"
image_file = image_paths[0]

answer = predict(question,image_file)
print(answer)

/home/yw699/anaconda3/envs/semantic_uncertainty/lib/python3.11/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Expanding inputs for image tokens in LLaVa should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


ER:  
Tell me more about this picture? ASSISTANT: The image features a beautiful butterfly perched on a purple flower. The butterfly is positioned in the center of the scene, with its wings spread out, showcasing its vibrant colors. The purple flower serves as a stunning backdrop for the butterfly, highlighting its beauty. The scene captures the delicate and intricate details of the butterfly and the flower, creating a captivating and serene moment.


In [7]:
from PIL import ImageOps
def predict(question, image_file, examples=None):
    """
    Few-shot 支持的预测函数
    :param question: 用户的问题
    :param image_file: 输入图像的 URL
    :param examples: Few-shot 示例，格式为 [{'question': '示例问题', 'image_url': '示例图像URL', 'answer': '示例答案'}, ...]
    :return: 生成的回答
    """
    # 初始化对话模板
    conversation = []

    # 如果有 few-shot 示例，添加到 conversation
    if examples:
        for example in examples:
            conversation.append({
                "role": "user",
                "content": [
                    {"type": "text", "text": example["question"]},
                    {"type": "image"},
                ],
            })
            conversation.append({
                "role": "assistant",
                "content": [
                    {"type": "text", "text": example["answer"]},
                ],
            })

    # 添加当前输入问题和图像
    conversation.append({
        "role": "user",
        "content": [
            {"type": "text", "text": question},
            {"type": "image"},
        ],
    })

    # 构造 prompt
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    

    # 加载图像
    raw_image = Image.open(requests.get(image_file, stream=True).raw)
    
    processed_image = ImageOps.fit(raw_image, (224, 224), Image.Resampling.LANCZOS)
    inputs = processor(images=processed_image, text=prompt, return_tensors='pt').to(0, torch.float16)
    # 模型生成
    output = model.generate(**inputs, max_new_tokens=200, do_sample=False)

    # 解码生成的答案
    answer = processor.decode(output[0][2:], skip_special_tokens=True)
    return answer


In [8]:
few_shot_examples = [
    {
        "question": "What is this object in the image?",
        "image_url": "https://m.media-amazon.com/images/I/61nv-RwdxsL.jpg",
        "answer": "OK.This is a coffee mug.",
    },
    {
        "question": "Can you identify the object in the picture?",
        "image_url": "https://pisces.bbystatic.com/image2/BestBuy_US/images/products/7cf25ee6-fa36-461a-86e8-011f85a06271.jpg;maxHeight=640;maxWidth=550;format=webp",
        "answer": "OK.This is a laptop.",
    },
]
# 定义问题和图像 URL
question = "What is the object in the picture?"
image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"

# 调用 predict 函数
response = predict(question, image_url, examples=few_shot_examples)
print("Generated Answer:", response)


ValueError: Image features and image tokens do not match: tokens: 1728, features 576